# taq_self_response_day_event_shift_data

#### Juan Camilo Henao Londono - 08.07.2019
#### AG Guhr - Universitaet Duisburg-Essen

In the following [link]() can be seen the plot of the function's results

In [1]:
# Modules

import numpy as np
import os

import pickle

import taq_data_tools

__tau__ = 1000

In [2]:
def taq_self_response_day_event_shift_data(ticker, date, tau):
    """
    Obtain the self response function using the midpoint price returns
    and trade signs of the ticker during different time lags. Return an
    array with the self response for a day and an array of the number of
    trades for each value of tau.
        :param ticker: string of the abbreviation of the midpoint stock to
         be analized (i.e. 'AAPL')
        :param date: string with the date of the data to be extracted
         (i.e. '2008-01-02')
    """

    date_sep = date.split('-')

    year = date_sep[0]
    month = date_sep[1]
    day = date_sep[2]

    function_name = taq_self_response_day_event_shift_data.__name__
    taq_data_tools.taq_function_header_print_data(function_name, ticker,
                                                  ticker, year, month, day)

    try:

        # Load data
        midpoint = pickle.load(open(''.join((
                '../../taq_data/article_reproduction_data_{1}/taq_midpoint'
                + '_full_time_data/taq_midpoint_full_time_data_midpoint_{1}'
                + '{2}{3}_{0}.pickle').split())
                .format(ticker, year, month, day), 'rb'))
        time_t, _, trade_sign = pickle.load(open("".join((
                '../../taq_data/responses_event_shift_data_{1}/taq_trade'
                + '_signs_responses_event_shift_data/taq_trade_signs'
                + '_responses_event_shift_data_{1}{2}{3}_{0}.pickle')
                .split())
                .format(ticker, year, month, day), 'rb'))
        # As the data is loaded from the original reproduction data from the
        # article, the data have a shift of 1 second. To correct this I made
        # both data to have the same time [34801, 56999]
        midpoint = midpoint[1:]
        time_m = np.array(range(34801, 57000))

        assert not np.sum(trade_sign == 0)
        assert not np.sum(midpoint == 0)

        # Array of the average of each tau. 10^3 s used by Wang
        shift_val = range(- 10 * tau, 10 * tau, 1)
        self_response_shift = np.zeros(len(shift_val))
        num = np.zeros(len(shift_val))

        # Calculating the midpoint log return and the self response function

        midpoint_t = 0. * trade_sign

        for t_idx, t_val in enumerate(time_m):
            condition = time_t == t_val
            len_c = np.sum(condition)
            midpoint_t[condition] = midpoint[t_idx] * np.ones(len_c)

        assert not np.sum(midpoint_t == 0)

        # Depending on the shift time value
        for s_idx, s_val in enumerate(shift_val):

            if (s_val < 0):
                midpoint_shift = midpoint_t[np.abs(s_val):]
                trade_sign_shift = trade_sign[:-np.abs(s_val)]

            elif (s_val > 0):
                midpoint_shift = midpoint_t[:-s_val]
                trade_sign_shift = trade_sign[s_val:]

            else:
                midpoint_shift = midpoint_t
                trade_sign_shift = trade_sign

            trade_sign_tau = trade_sign_shift[:-tau - 1]
            trade_sign_no_0_len = len(trade_sign_tau[trade_sign_tau != 0])
            num[s_idx] = trade_sign_no_0_len
            # Obtain the midpoint log return. Displace the numerator tau
            # values to the right and compute the return

            # Midpoint price returns
            log_return_sec = (midpoint_shift[tau + 1:]
                              - midpoint_shift[:-tau - 1]) \
                / midpoint_shift[:-tau - 1]

            # Obtain the self response value
            if (trade_sign_no_0_len != 0):
                product = log_return_sec * trade_sign_tau
                self_response_shift[s_idx] = np.sum(product)

        return self_response_shift, num

    except FileNotFoundError as e:
        print('No data')
        print(e)
        print()
        return None

In [3]:
# Test

tickers = ['AAPL', 'MSFT']
dates = []

year = '2008'
month = '03'
days = ['10', '11', '12', '13', '14']

tau = 1

for day in days:
    dates.append(year + '-' + month + '-' + day)

for ticker in tickers:
    for date in dates:
        taq_self_response_day_event_shift_data(ticker, date, tau)

TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock AAPL the 2008.03.10
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock AAPL the 2008.03.11
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock AAPL the 2008.03.12
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock AAPL the 2008.03.13
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock AAPL the 2008.03.14
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock MSFT the 2008.03.10
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock MSFT the 2008.03.11
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock MSFT the 2008.03.12
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock MSFT the 2008.03.13
TAQ data
taq_self_response_day_event_shift_data
Processing data for the stock MSFT the 2008.03.14
